# # This notebook is to detect building structures by object detection to cleanup the data for training

In [1]:
import pandas as pd
import os
import shutil
import cv2
from multiprocessing import Pool
import time
import gc
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import tensorflow as tf
import PIL
from PIL import Image
import imageio
import logging as log
import tarfile
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
!pwd

/cygdrive/c/Apps/jupyterWorkspace/google_landmark_detection/Models


http://download.tensorflow.org/models/object_detection/ssd_resnet101_v1_fpn_shared_box_predictor_oid_512x512_sync_2019_01_20.tar.gz

In [4]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
        fname=model_name,
        origin=base_url + model_file,
        untar=True)
    print(model_dir)
    model_dir = pathlib.Path(model_dir)/"saved_model"
    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [5]:
# model_path = "/google_landmark_detection/datasets/OpenImagesDatasetV6/facessd_mobilenet_v2_quantized_320x320_open_image_v4/"
model_name = 'ssd_resnet101_v1_fpn_shared_box_predictor_oid_512x512_sync_2019_01_20'
detection_model = load_model(model_name)

C:\Apps\cygwin\home\adrsi\.keras\datasets\ssd_resnet101_v1_fpn_shared_box_predictor_oid_512x512_sync_2019_01_20
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [6]:
print(detection_model.inputs)
print(detection_model.output_dtypes)
print(detection_model.outputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]
{'detection_scores': tf.float32, 'detection_classes': tf.float32, 'num_detections': tf.float32, 'detection_boxes': tf.float32}
[<tf.Tensor 'detection_boxes:0' shape=(None, 100, 4) dtype=float32>, <tf.Tensor 'detection_classes:0' shape=(None, 100) dtype=float32>, <tf.Tensor 'detection_scores:0' shape=(None, 100) dtype=float32>, <tf.Tensor 'num_detections:0' shape=(None,) dtype=float32>]


In [8]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                   for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            output_dict['detection_masks'], output_dict['detection_boxes'],
            image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

In [9]:
def show_inference(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=8)

    display(Image.fromarray(image_np))

In [37]:
image_path = 'C:\\Apps\\jupyterWorkspace\\google_landmark_detection\\datasets\\group1_set_128\\set_128\\train\\27\\0fa907fb215b2787.jpg'
show_inference(detection_model, image_path)

NameError: name 'vis_util' is not defined

In [34]:
print(image_path)

C:\Apps\jupyterWorkspace\google_landmark_detection\datasets\group1_set_128\set_128	rain fa907fb215b2787.jpg


In [ ]:
Facessd_Mobilenet_V2_Quantized_Open_Image_V4=tf.keras.applications..(input_shape=(128,128,3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
len(Inception_ResNetV2_MODEL.layers)

In [ ]:
Inception_ResNetV2_MODEL.trainable = False

In [ ]:
# Inception_ResNetV2_MODEL.layers[107].name

In [ ]:
# for layer in Inception_ResNetV2_MODEL.layers[:107]:
#     layer.trainable=False

In [ ]:
Inception_ResNetV2_MODEL.summary()

In [ ]:
model=tf.keras.models.Sequential([
                                  Inception_ResNetV2_MODEL,
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(512, activation='relu'),
                                  tf.keras.layers.Dense(92740, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
!pwd

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=50,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

#flow training images in batches of 128
train_generator = train_datagen.flow_from_directory(
    '../datasets/set_128/train/',
    target_size=(128,128),
    batch_size=256,
    class_mode='categorical'
)
history = model.fit_generator(
    train_generator,
    epochs = 30,
    verbose=1
)

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(len(acc))

plt.plot(epochs, acc)
plt.title('Training accuracy')
plt.figure()

plt.plot(epochs, loss)
plt.title('Training Loss')
plt.figure()